In [167]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score




In [168]:
url = "https://drive.google.com/file/d/14nr7lkB1_fF61Mba9mNgTnCeFlxBATnt/view?usp=sharing" # spotify list
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]

songs_original_df = pd.read_csv(path)
songs_original_df.columns = songs_original_df.columns.str.replace(' ', '')   #replace the space in front of the columns name
to_drop=['Unnamed:0']               #drop this useless columns, needed to get rid of duplicates

songs_original_df = songs_original_df.drop(columns=to_drop)  
songs_original_df = songs_original_df.drop_duplicates()         #drop duplicates

to_drop=[ 'key',                        #drop addtl. useless columns
          'mode',
          'type',
          'duration_ms',
          'time_signature'
          #'id',
          #'html',
          #'artist',
          #'liveness',
          #'speechiness',
          #'loudness',
          #'energy',
         # 'acousticness',
          #'instrumentalness',
          #'valence',
          #'tempo',
          #'danceability'
    
         ]
 
songs_original_df = songs_original_df.drop(columns=to_drop)  
songs_original_df= songs_original_df.set_index(['name', 'html']) # set the index to name and html to keep html it accessible
songs_original_df = songs_original_df.drop_duplicates()   

songs_original_df

0


,,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
name,html,,,,,,,,,,,
Se Eu Quiser Falar Com Deus,https://open.spotify.com/track/1n7JnwviZ7zf0LR1tcGFq7,Gilberto Gil,0.6580,0.25900,-13.141,0.0705,0.694,0.000059,0.9750,0.3060,110.376,1n7JnwviZ7zf0LR1tcGFq7
Saudade De Bahia,https://open.spotify.com/track/5QGM1U0eCYrQuwSJwTm5Zq,Antônio Carlos Jobim,0.7420,0.39900,-12.646,0.0346,0.217,0.000002,0.1070,0.6930,125.039,5QGM1U0eCYrQuwSJwTm5Zq
"Canta Canta, Minha Gente",https://open.spotify.com/track/0NLIFSZxPzQhCwnkn5PJYs,Martinho Da Vila,0.8510,0.73000,-11.048,0.3470,0.453,0.000063,0.1240,0.9050,93.698,0NLIFSZxPzQhCwnkn5PJYs
Mulher Eu Sei,https://open.spotify.com/track/3mXqOdlLE1k67WsAxryPFs,Chico César,0.7050,0.05020,-18.115,0.0471,0.879,0.000041,0.3860,0.5240,106.802,3mXqOdlLE1k67WsAxryPFs
Rosa Morena,https://open.spotify.com/track/7bSzjzjTkWT2CkIPPdp0eA,Kurt Elling,0.6510,0.11900,-19.807,0.0380,0.916,0.000343,0.1040,0.4020,120.941,7bSzjzjTkWT2CkIPPdp0eA
...,...,...,...,...,...,...,...,...,...,...,...,...
"1812 Festival Overture, Op. 49: 1812 Overture, Op. 49",https://open.spotify.com/track/1aEhYlZtypmipA06SDJ4U3,Pyotr Ilyich Tchaikovsky,0.2020,0.06940,-23.390,0.0473,0.917,0.715000,0.0832,0.0655,87.906,1aEhYlZtypmipA06SDJ4U3
"Winter Fragments pour ensemble instrumental, sons de synthèse et dispositif électronique",https://open.spotify.com/track/1Gfqe7TAKklmuZf6hxsH6h,Tristan Murail,0.1880,0.10100,-21.873,0.0442,0.855,0.426000,0.0694,0.0355,83.134,1Gfqe7TAKklmuZf6hxsH6h
"Schoenberg: 5 Orchestral Pieces, Op. 16: No. 3 Farben (Colours)",https://open.spotify.com/track/2XNwnFrdMDpismp0VUZ7cU,Arnold Schoenberg,0.0596,0.00093,-42.959,0.0434,0.951,0.969000,0.0772,0.0344,71.573,2XNwnFrdMDpismp0VUZ7cU


In [169]:
songs_original_df.nunique() #id should be unique

artist              2176
danceability         882
energy              1191
loudness            4310
speechiness         1001
acousticness        2545
instrumentalness    2168
liveness            1128
valence             1267
tempo               4824
id                  5171
dtype: int64

Min_Max_Scaler

In [170]:
songs = songs_original_df.copy()
to_scale = ['acousticness', 'instrumentalness','danceability','energy','loudness','speechiness','liveness','valence','tempo']  # scale all features which might be interesting for clustering
to_cluster = ['acousticness', 'instrumentalness'] #these are the only columns which will be clustered via KMEANS below

# Initialise the transformer (optionally, set parameters)
my_min_max = MinMaxScaler().set_output(transform="pandas")

# Use the transformer to transform the data
songs[to_scale]= my_min_max.fit_transform(songs[to_scale])

# [['danceability','energy','loudness','acousticness','instrumentalness']]
songs

,,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
name,html,,,,,,,,,,,
Se Eu Quiser Falar Com Deus,https://open.spotify.com/track/1n7JnwviZ7zf0LR1tcGFq7,Gilberto Gil,0.680455,0.25900,0.763897,0.076797,0.696787,0.000060,0.987842,0.310660,0.515800,1n7JnwviZ7zf0LR1tcGFq7
Saudade De Bahia,https://open.spotify.com/track/5QGM1U0eCYrQuwSJwTm5Zq,Antônio Carlos Jobim,0.767322,0.39900,0.771967,0.037691,0.217871,0.000002,0.108409,0.703553,0.584322,5QGM1U0eCYrQuwSJwTm5Zq
"Canta Canta, Minha Gente",https://open.spotify.com/track/0NLIFSZxPzQhCwnkn5PJYs,Martinho Da Vila,0.880041,0.73000,0.798018,0.377996,0.454819,0.000064,0.125633,0.918782,0.437862,0NLIFSZxPzQhCwnkn5PJYs
Mulher Eu Sei,https://open.spotify.com/track/3mXqOdlLE1k67WsAxryPFs,Chico César,0.729059,0.05020,0.682811,0.051307,0.882530,0.000041,0.391084,0.531980,0.499098,3mXqOdlLE1k67WsAxryPFs
Rosa Morena,https://open.spotify.com/track/7bSzjzjTkWT2CkIPPdp0eA,Kurt Elling,0.673216,0.11900,0.655228,0.041394,0.919679,0.000348,0.105370,0.408122,0.565171,7bSzjzjTkWT2CkIPPdp0eA
...,...,...,...,...,...,...,...,...,...,...,...,...
"1812 Festival Overture, Op. 49: 1812 Overture, Op. 49",https://open.spotify.com/track/1aEhYlZtypmipA06SDJ4U3,Pyotr Ilyich Tchaikovsky,0.208893,0.06940,0.596818,0.051525,0.920683,0.725888,0.084296,0.066497,0.410795,1aEhYlZtypmipA06SDJ4U3
"Winter Fragments pour ensemble instrumental, sons de synthèse et dispositif électronique",https://open.spotify.com/track/1Gfqe7TAKklmuZf6hxsH6h,Tristan Murail,0.194416,0.10100,0.621548,0.048148,0.858434,0.432487,0.070314,0.036041,0.388495,1Gfqe7TAKklmuZf6hxsH6h
"Schoenberg: 5 Orchestral Pieces, Op. 16: No. 3 Farben (Colours)",https://open.spotify.com/track/2XNwnFrdMDpismp0VUZ7cU,Arnold Schoenberg,0.061634,0.00093,0.277803,0.047277,0.954819,0.983756,0.078217,0.034924,0.334469,2XNwnFrdMDpismp0VUZ7cU


KMEANS

In [179]:
k_seed=123
my_kmeans = KMeans(n_clusters= 38, # you always choose the number of k here
                   n_init = "auto",
                   random_state = k_seed)

# Fit the model to the data
my_kmeans.fit(songs[to_cluster])

# Obtain the cluster output
clusters = my_kmeans.labels_

# Attach the cluster output to our original DataFrame
songs["playlist"] = clusters

#min_max_df.loc[min_max_df['playlist']==7,:]
songs[[to_cluster[0],to_cluster[1],'playlist']].sample(20)

,,acousticness,instrumentalness,playlist
name,html,,,
The Faces Right Below the Skin of the Earth,https://open.spotify.com/track/78Il4uvXABzhLjtSkXvlHf,0.000002,0.963452,20
All Alone,https://open.spotify.com/track/6TAW00MAPvS59yEoIgtOEI,0.986948,0.906599,30
Airplanes (feat. Hayley Williams of Paramore),https://open.spotify.com/track/6lV2MSQmRIkycDScNtrBXO,0.110442,0.000000,26
Song for Three,https://open.spotify.com/track/38JLOKhjwtBuA8VtsWda1M,0.946787,0.881218,30
Baal Reginon,https://open.spotify.com/track/4CtGvjULcucdLKNsaA0Dfr,0.000018,0.815228,4
Hurt,https://open.spotify.com/track/6VMLTjHtZc6QPt0RcWPUOJ,0.931727,0.919797,30
Gimme Hope Joachim,https://open.spotify.com/track/2o9mLr7KuxCs2nY84IyQDS,0.735944,0.000000,3
Desperate Cry,https://open.spotify.com/track/3PWgD7Gk14tGxHIEK8PQ6o,0.000008,0.163452,9
Flower Through Concrete,https://open.spotify.com/track/5wHT1UYWQBHYAeIwRbBDJG,0.990964,0.904569,30


In [172]:
songs.groupby('playlist')['artist'].count() #songs per playlist

playlist
0      99
1     397
2     256
3     135
4     139
5     137
6      81
7      42
8     136
9      98
10    148
11     59
12     15
13    245
14     26
15     22
16     16
17    920
18     76
19     14
20    225
21    131
22    143
23    104
24     31
25    136
26    322
27     22
28     14
29     62
30    151
31     86
32     17
33    203
34     19
35    118
36    150
37    176
Name: artist, dtype: int64

In [191]:
output = songs.loc[songs['playlist']==20, [to_cluster[0],to_cluster[1]]
                    ].sample(40)

output
               

,,acousticness,instrumentalness
name,html,,
The Corpsegrinder Experience,https://open.spotify.com/track/5xSmXzRXwiwnmzfcqiHiDO,0.000001,0.934010
Eternia - 2020 Remastered,https://open.spotify.com/track/53OL9llnm9ZkK0Kar0ONCb,0.000006,0.948223
Torn Apart,https://open.spotify.com/track/04ugH5Ht77qUXdx9Yk6nAJ,0.000030,0.919797
The Krusher,https://open.spotify.com/track/6YE7EUgXnpK5wgWy9mGPRX,0.000114,0.929949
Mass Appeal Madness,https://open.spotify.com/track/0d2yocMUubmC5qL15AZzIF,0.000071,0.953299
Four Walls,https://open.spotify.com/track/0KPFDdW6AplKCH30dxP79J,0.000023,0.975635
Xeper,https://open.spotify.com/track/4onr5W7mLre4pB3o7dRe43,0.000004,0.921827
Storm Of Stress,https://open.spotify.com/track/2SduZ4qG5P63L2e1Z038es,0.000291,0.939086
"No Guts, No Glory",https://open.spotify.com/track/5tUxhJPbbOB4vEwm7ax1m2,0.000001,0.948223


In [180]:
new_df  = songs.groupby('playlist')[to_scale].mean()

In [181]:
new_df['sum'] = new_df[to_cluster[0]]+ new_df[to_cluster[1]]   # Piano Relax
new_df.sort_values(by='sum', ascending=False).head(2)

,acousticness,instrumentalness,danceability,energy,loudness,speechiness,liveness,valence,tempo,sum
playlist,,,,,,,,,,
2,0.980430,0.944777,0.332825,0.089938,0.587833,0.051789,0.117832,0.147905,0.482620,1.925207
30,0.972267,0.884136,0.344849,0.139419,0.624589,0.046983,0.139323,0.193290,0.495611,1.856403


In [176]:
new_df['zero_one'] = new_df[to_cluster[0]] - new_df[to_cluster[1]]
new_df.sort_values(by='zero_one', ascending=False).head(2)

,acousticness,instrumentalness,danceability,energy,loudness,speechiness,liveness,valence,tempo,sum,zero_one
playlist,,,,,,,,,,,
10,0.954935,0.009866,0.428443,0.172336,0.719401,0.050166,0.169965,0.278122,0.515511,0.964801,0.945068
25,0.852653,0.003741,0.567614,0.339632,0.780096,0.062538,0.182370,0.512652,0.526376,0.856394,0.848912


In [177]:
new_df['one_zero'] =  new_df[to_cluster[1]]-new_df[to_cluster[0]]
new_df.sort_values(by='one_zero', ascending=False).head(2)

,acousticness,instrumentalness,danceability,energy,loudness,speechiness,liveness,valence,tempo,sum,zero_one,one_zero
playlist,,,,,,,,,,,,
20,0.006130,0.943625,0.277803,0.891849,0.834141,0.108540,0.201742,0.24696,0.580079,0.949756,-0.937495,0.937495
33,0.003003,0.890230,0.296756,0.893621,0.842973,0.104415,0.216370,0.25968,0.568739,0.893233,-0.887227,0.887227
